In [14]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType
from pyspark.sql import DataFrameWriter


from ast import literal_eval
from pyspark.sql.types import StructField, StructType, StringType, DoubleType, TimestampType

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [15]:
from pyspark.sql.functions import from_json

In [16]:
from pyspark.sql import SparkSession

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [18]:
spark = SparkSession\
.builder\
.appName("read")\
.master("local[4]")\
.config("spark.jars", "/spark-3.0.1-bin-hadoop2.7/jars/postgresql-42.2.5.jar")\
.getOrCreate()

In [19]:
spark

In [20]:
tweets=spark.readStream \
  .format("kafka")\
  .option("kafka.bootstrap.servers", "13.74.113.150:9092") \
  .option("subscribe", "benten123") \
  .option("header", True)\
  .option("driver", "org.postgresql.Driver")\
  .load()\
  .selectExpr("cast(value as string) as value","timestamp")\



In [21]:
tweets.isStreaming

True

In [22]:
counting=tweets.withWatermark("timestamp", "2 minutes").groupBy("value", window("timestamp", "1 minutes")).count()

In [23]:
#Counting 2, table small
counting2=tweets.groupBy("value").agg(count("value").alias("count"))

In [24]:
def postgres_sink(data_frame, epoch_id): #that
    dbname = ('twitter')
    dbuser = ('postgres')
    dbpass = ('myPassword')
    dbhost = ('localhost')
    dbport = ('5432')

    url = "jdbc:postgresql://"+dbhost+":"+dbport+"/"+dbname
    properties = {
        "driver": "org.postgresql.Driver",
        "user": dbuser,
        "password": dbpass
    }

    data_frame.write.jdbc(url=url, table="stream", mode="overwrite",
                          properties=properties)

In [25]:
#DO NOT FORG.
counting=counting.selectExpr("value", "CAST(window AS STRING)", "count")

In [26]:
#Sink to Postgres with Foreachbatch
easy = counting.writeStream.trigger(processingTime='1 minute').foreachBatch(postgres_sink).outputMode("complete").start() #THIS

22/01/19 18:30:40 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-871eba7d-9a3e-4e73-8589-12465ca4ccd2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
